As we investigated the AIS ship tracking data, we became interested in automatically detecting emergent behavior from groups of ships. For instance: can we automatically detect container ships following a shipping lane? can we find groups of ships moving together in a convoy, or a fishing fleet working together?

We began investigating traffic from our ETL-ed `AIS` data from January 1, 2015. The ships are all centered off the coast of Alaska, near the Aleutian Islands. First, we wanted an overview of the average ship position each hour, to get a sense of the data:

![all_vessels](img/colored_by_ship.png)

Near the end of the penninsula, seen in our map as the grey trapezoid on the right hand side, we can see a significant amount of ships all following the same path. Investigation of the ships following this path reveals that they are primarily large container and tanker vessels, indicating a shipping lane and a good target for testing our shipping lane identification algorithms.

In the southwestern corner of the map, we see several ships moving along approximately the same course. After direct examination, we identify that the two ships represented by the blue and yellow markers, the `Gulf Valour` and the `Pole`, travel at the same rate alongside each other throughout our sample day. This is the exact type of feature we want for identification of convoys, making this a good test candidate.

Finding groups in large datasets such as this is a great application of clustering analysis. Here, we leveraged `HDBSCAN`, developed by [Campello et al.](https://hdbscan.readthedocs.io/en/latest/how_hdbscan_works.html) in 2013. `HDBSCAN` improves upon the widely used density-based clustering algorithm `DBSCAN` by turning it into a hierarchical clustering algorithm, which allows it to discover clusters of varying densities within a dataset. This algorithm has a distinct advantage over more common methods such as k-means clustering, as the analyst does not need to specifiy the number of clusters before. Additionally, it can find clusters with more complex shapes and varying densities.

![kmeans_hdbscan_comparison](img/kmeans_hdbscan_comparison.png)

As a first pass at clustering, we directly used the ship positions over the course of the day. For each hour, we found the average position (lat-lon) of each ship (only when it checked in during that hour). We ended up with a dataset that looked like:

![df_no_bearing](img/df_no_bearing.png)

We then used the lattitude and longitude of the ship during each hour to cluster ships together. Because we subset each position by hour, each ship will appear in a cluster each hour that it is present. This means that ships traveling along the same route at different times can end up in the same cluster. When this occurs for a large number of ships across differeing hour intervals, we have a strong indication for a shipping lane.

<!-- ![positions](img/position_based_clusters.png) -->
<img src="img/position_based_clusters.png" alt="position_based_clusters" style="width: 900px;"/>

After clustering only based on position, we see that we do find geographically similar groupings of ships. However, the shipping lane near the edge of the penninsula consists of many different cluster groupings, and many of the ships outside of the shipping lane also appear in the same clusters. We haven't really distinguished the types of activity we want, but it's a good start.

For the next iteration, we computed the average ship bearing for each time interval, using this as an additional feature. Ships traveling in the same shipping lane, or in a convoy, should often have the same approximate bearing. This, combined with the original lat-lon based proximity features, greatly improved the clustering. Here, we show the new set of clusters produced when we include bearing as a feature. Notice that the shipping lane just of the coast now becomes very distinct and well clustered.

<!-- ![velocities_all](img/velocity_clusters_all.png) -->
<img src="img/velocity_clusters_all.png" alt="velocity_clusters_all" style="width: 900px;"/>

For simplicity, we then look at only the top few clusters of interest. The shipping lane comes out as a strong blue line, thanks to the 18 ships that make up that cluster over the 24 hours of data present for this clustering analysis. We can also see additional patterns becoming more obvious now as well. The red line of 14 separate ships traveling northeast towards the tip of the penninsula, or the light green cluster showing the travel of the ship `Malalo`. The fairly consistent bearing and nearby position allows the clustering algorithm to identify the track taken by this ship even with a 10 hour gap in AIS data for this ship as it travels northwest. Additionally, we now see that the `Gulf Valour` and `Pole` have become a single cluster, getting us closer to our goal of convoy identification.

<!-- ![velocities](img/velocity_clusters.png) -->
<img src="img/velocity_clusters_subset.png" alt="velocity_clusters_subset" style="width: 900px;"/>

In the future, we want to move towards performing clustering each hour, and identifying groups of ships that consistently appear in the same cluster together across time. This is a solid indication of shared activity patterns such as convoys.